In [ ]:
#git clone https://github.com/lucidrains/iTransformer.git

In [1]:
# TODO which one?
#import iTransformer
import sys
sys.path.append('/vol/fob-vol7/nebenf21/reinbene/bene/MA/iTransformer') 
from iTransformer import iTransformer
import torch
from pathlib import Path

import torch
import torch.optim as optim
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter

from utils import data_handling, helpers
import config
print("Import succesfull")


Import succesfull


In [2]:
window_size = 96
pred_length = 96
dataset_size = 1.0 # float as percentage

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

electricity_dict = data_handling.load_electricity()
dataloader_train, dataloader_validation, dataloader_test = data_handling.convert_data(electricity_dict, window_size, pred_length)

Using device: cuda
Feature batch shape: torch.Size([32, 96, 348])
Labels batch shape: torch.Size([32, 96, 348])


In [5]:
model_config = {
    'num_variates': electricity_dict["train"].size(1),
    'lookback_len': window_size,
    'depth': 6,
    'dim': 512,
    'num_tokens_per_variate': 1,
    'pred_length': pred_length,
    'dim_head': 32,
    'heads': 4,
    'attn_dropout': 0.1,
    'ff_mult': 4,
    'ff_dropout': 0.1,
    'num_mem_tokens': 4,
    'use_reversible_instance_norm': True,
    'reversible_instance_norm_affine': False,
    'flash_attn': True
}

# defining all needed instances
model = iTransformer(**model_config).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)
writer = SummaryWriter(log_dir=config.CONFIG_LOGS_PATH["revin"])


Non-A100 GPU detected, using math or mem efficient attention if input tensor is on cuda


In [6]:
epoch = 1

def train_one_epoch(epoch, model, device, dataloader_train, optimizer, scheduler, writer):
	global_step = 0
	model.train()
	total_loss = 0
	for input, target in tqdm(dataloader_train, desc=f"Epoch: {epoch}"):
		optimizer.zero_grad()
		input = input.to(device)
		target = target.to(device)
		loss = model(input, target)
		
		loss.backward()
		optimizer.step()
		total_loss += loss.item()

		writer.add_scalar('train_loss', loss, global_step)
		writer.add_scalar('train_lr', optimizer.param_groups[0]['lr'], global_step)
		lr =  optimizer.param_groups[0]['lr']
		global_step+=1

	print(f'Epoch {epoch}, Loss: {total_loss / len(dataloader_train)}, LR: {lr}')

	scheduler.step()

	writer.close()
	if epoch % 5 == 0:
		helpers.create_checkpoint(model, optimizer, scheduler, epoch, loss, global_step, "iTransfomrer_electricity")

for epoch in range(1, epoch + 1):
	train_one_epoch(epoch, model, device, dataloader_train, optimizer, scheduler, writer)


Epoch: 1:  14%|█▍        | 24/170 [00:05<00:32,  4.49it/s]


KeyboardInterrupt: 

In [31]:
def eval_function(valid_dataloader):
	if model == None:
		model = iTransformer(**config)
		checkpoint = torch.load("/vol/fob-vol7/nebenf21/reinbene/bene/MA/outputs/series_standardized/model_first_20_ids_full_epoch_10.pt")
		model.load_state_dict(checkpoint['model_state_dict'])
		optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
		start_epoch = checkpoint['epoch'] + 1  # Start training from 
		scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
		scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
		writer = SummaryWriter(log_dir='logs')
		global_step = checkpoint['global_step_writer']
		device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
		model.to(device)

	model.eval()
	global_step_eval = 1
	total_p10_eval = 0
	total_p50_eval = 0
	total_p90_eval = 0

	# Example inference code
	with torch.no_grad():
		total_loss_eval = 0
		for input, target in tqdm(valid_dataloader, desc=f"Epoch: Validating"):
			input = input.to(device)
			target = target.to(device)
			output = model(input)

			loss = torch.nn.MSELoss()
			loss = loss(output[96], target)  # compute loss on all variates
			#computed_loss = loss(output, target)  # compute loss on target variate

			total_loss_eval += loss.item()
			#total_p10_eval += calc_percentiles(output[12].cpu(), target.cpu(), 10)
			#total_p50_eval += calc_percentiles(output[12].cpu(), target.cpu(), 50)
			#total_p90_eval += calc_percentiles(output[12].cpu(), target.cpu(), 90)


	print(f'MSE Loss: {total_loss_eval / len(valid_dataloader)}')#\nP10: { total_p10_eval/ len(valid_dataloader)}\n\
	#P50: {total_p50_eval/ len(valid_dataloader)}\nP90: { total_p90_eval/ len(valid_dataloader)}')

	#writer.add_scalar('MSE/valid', total_loss_eval/ len(valid_dataloader), global_step_eval)
	#writer.add_scalar('P10/valid', total_p10_eval/ len(valid_dataloader), global_step_eval)
	#writer.add_scalar('P50/valid', total_p50_eval/ len(valid_dataloader), global_step_eval)
	#writer.add_scalar('P90/valid', total_p90_eval/ len(valid_dataloader), global_step_eval)

	writer.close()

In [22]:
eval_function(dataloader_validation)

Epoch: Validating: 100%|██████████| 1/1 [00:00<00:00, 27.98it/s]

MSE Loss: 0.1835039108991623


In [32]:
eval_function(dataloader_test)

Epoch: Validating: 100%|██████████| 86/86 [00:04<00:00, 19.86it/s]

MSE Loss: 0.1695066345639007
